In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn
import math
import numpy as np
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.preprocessing import StandardScaler, PowerTransformer
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import FeatureUnion, Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score
from scipy import stats
import itertools as it
import sklearn.metrics as metrics
import sklearn.model_selection as selection
import os
import json
import pickle as pic
import importlib
import sys
sys.path.append("./../utils")
sys.path.append("./../sklearn")
import util
import constants as const
import ColumnTransformer
importlib.reload(util)
importlib.reload(const)
importlib.reload(ColumnTransformer)
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)
pd.set_option('max_rows', None)
pd.set_option('precision', 6)
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
#from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBRegressor
import json
import pickle
import helper

# 0. Some Constants and Setup

In [2]:
paper_partitioners = [
    '1dd', 
    '1ds', 
    '2d', 
    '2ps', 
    'crvc', 
    'dbh', 
    'hdrf',
    'hep1',
    'hep10', 
    'hep100', 
    'ne',
] 


TARGET = "processing_time"
target = "processing_time"

# 1. Load Data

In [3]:
# Yes, we make copies. We will later filter. 
X_train = pd.read_csv("../datasets/graph-processing-run-time_train.csv")
y_train = X_train.copy()
# Yes, we make copies. We will later filter. 
X_test = pd.read_csv("../datasets/graph-processing-run-time_test.csv")
y_test = X_test.copy()

# 2. Helper
## 2.1 For Evaluation

In [4]:
def evaluate_model(model_name, algorithm, model, X_test, y_test, target):
    """ We evalute the given model on the testset. 

    Args:
        model_name (string): the name of the model, e.g., XGB for extrem gradient boosting
        algorithm (string): the graph processing algorithm, e.g., cc, pr, sssp1, ...
        model (model): the trained model
        X_test (test data features): the testdata
        y_test (test date target): the testdata
        features (list): the features used in the model
        target (string): the name if the target column
    Returns:
        list of dicts: the scores evaluated by partitioner and by graph 
    """
    
    graphs = X_test.graph.unique()
    partitioners = X_test.partitioner.unique()

    results = []
    # 1. Get scores by partitioner and per test graph. This for loop also works if we only have on partitioner.
    for graph in graphs:
        for partitioner in partitioners:
            _X_test = X_test.copy()
            _y_test = y_test.copy()
            
            # Only evaluate the current graph on the current partitioner
            _X_test = _X_test[(_X_test.graph == graph)  & (_X_test.partitioner == partitioner) ]
            _y_test = _y_test[(_y_test.graph == graph) & (_y_test.partitioner == partitioner)]

            # The predicted processing time
            _X_test["predicted_{}".format(target)] = model.predict(_X_test)

            # The true processing time
            _X_test[target] = _y_test[target]

            # Get the metrics/scores.
            metrics = helper.Metrics(target, "predicted_{}".format(target))
            result = metrics.get_metrics(_X_test)
            
            result["partitioner"] = partitioner
            result["graph"] = graph.split("/")[-1]
            result["model"] = model_name
            result["algorithm"] = algorithm
            results.append(result)

    # Since we also use this evaluation method for a model trained only for one single partitioner. 
    # In the next line we will evaluate how well the model performed for the different partitioners on the given graph processing algorithm: One model multiple partitioners and one graph processing algorithm.
    if (len(partitioners) > 1):
        # 2. Get result by partitioner and per test grap
        for partitioner in partitioners:
            _X_test = X_test.copy()
            _y_test = y_test.copy()
            
            # Only evaluate the current graph on the current partitioner
            _X_test = _X_test[(_X_test.partitioner == partitioner)]
            _y_test = _y_test[(_y_test.partitioner == partitioner)]

            # The predicted processing time
            _X_test["predicted_{}".format(target)] = model.predict(_X_test)

            # The true processing time
            _X_test[target] = _y_test[target]

            metrics = helper.Metrics(target, "predicted_{}".format(target))
            result = metrics.get_metrics(_X_test)

            result["partitioner"] = partitioner
            result["graph"] = "all"
            result["model"] = model_name
            result["algorithm"] = algorithm
            results.append(result)

    
        # 3. Get the overalle score for all partitoiners on all graphs for the fiven graph processing algorithm
        _X_test = X_test.copy()
        _y_test = y_test.copy()
    
        _X_test["predicted_{}".format(target)] = model.predict(_X_test)
        _X_test[target] = _y_test[target]


        metrics = helper.Metrics(target, "predicted_{}".format(target))
        result = metrics.get_metrics(_X_test)
        result["partitioner"] = "all"
        result["graph"] = "all"
        result["model"] = model_name
        result["algorithm"] = algorithm
        results.append(result)
    
    #IF We only train with one partitioner, then we can not set partitioner to "all"
    else:
        # 3. Get for the one partitioner the scores. In this else we only have one partitioner which will be at index 0
        partitioner = partitioners[0]
        _X_test = X_test.copy()
        _y_test = y_test.copy()
    
        _X_test["predicted_{}".format(target)] = model.predict(_X_test)
        _X_test[target] = _y_test[target]


        metrics = helper.Metrics(target, "predicted_{}".format(target))
        result = metrics.get_metrics(_X_test)
        result["partitioner"] = partitioner
        result["graph"] = "all"
        result["model"] = model_name
        result["algorithm"] = algorithm
        results.append(result)

    return results

# 2.2 For Training

In [5]:
def train(model_name,algorithm, model, X_train, y_train, X_test, y_test, feature_combinations, target, store_model=False, result_file_name=""):
    print("Trainer will store model in ", result_file_name)
    """Traing the given pipeline on the training data and evaluate the model on the test set. 

    Args:
        model_name (string): name of the model, e.g., rfr for random forest regressor
        model (pipeline): the pipeline to traing
        X_train (_type_): train data - features
        y_train (_type_): train data - target
        X_test (_type_): test data - features
        y_test (_type_): test data - target
        target (_type_): the target to predict

    Returns:
        list: the achieved scores
    """
    _X_train = X_train.copy()
    _y_train = y_train.copy()
    _X_test = X_test.copy()
    _y_test = y_test.copy()

    model.fit(_X_train, _y_train[target])
    print("PARAMETERS", model.get_params().keys())
    print("Best params", model.best_params_)
    print("Mean Test score", model.cv_results_["mean_test_score"])
    
    # Now we have the trained model
    # We store all the cross validation scores and also the best parameters
    # We further directly evaluation on the test set and also store these scores. 
    # IMPORTANT: Later on, we need to make the model selection based on the crossvalidation scores, not on the test set. 

    # We serialize and store the best model     
    if (store_model and not "" == result_file_name):
        pickle.dump(model, open(result_file_name, 'wb'))

    # Get the scores. 
    scores = evaluate_model(model_name, algorithm, model, X_test=_X_test, y_test=_y_test, target=target)
    # Yes, for each score we add the cv score on the rmat graphs, the best hyperparameters. 
    # Because even if we evaluate on different graphs/partitioners, these score do not differ because they are based on synthetic graphs and lead to the model selection  
    for score in scores:
        score["rmat_best_params"] = model.best_params_
        score["rmat_cv_score"] =  model.cv_results_["mean_test_score"][model.best_index_]
        score["rmat_cv_scores"] =  model.cv_results_["mean_test_score"]
        score["rmat_best_index"] = model.best_index_
    return scores
   # print(scores)

# 3. Training used features
## 3.1 Used features

In [6]:
graph_properties_simple = [
  'num_edges',
  'num_vertices',
]

featuresets = []
for graph_properties in [
  graph_properties_simple, 
  ]:
  quality_features = helper.create_Feature_combinations(
    [
      "vertex_balance",	
      "destination_balance",	
      "source_balance",	
      "replication_factor",
      "edge_balance",
    ], 
    [])   
  for quality_feature in quality_features:
    features = quality_feature + graph_properties
    featuresets.append(features)
print("Our features set consits of", len(featuresets), " combinations\nfeatures: ", featuresets)

Our features set consits of 32  combinations
features:  [['num_edges', 'num_vertices'], ['vertex_balance', 'num_edges', 'num_vertices'], ['destination_balance', 'num_edges', 'num_vertices'], ['vertex_balance', 'destination_balance', 'num_edges', 'num_vertices'], ['source_balance', 'num_edges', 'num_vertices'], ['vertex_balance', 'source_balance', 'num_edges', 'num_vertices'], ['destination_balance', 'source_balance', 'num_edges', 'num_vertices'], ['vertex_balance', 'destination_balance', 'source_balance', 'num_edges', 'num_vertices'], ['replication_factor', 'num_edges', 'num_vertices'], ['vertex_balance', 'replication_factor', 'num_edges', 'num_vertices'], ['destination_balance', 'replication_factor', 'num_edges', 'num_vertices'], ['vertex_balance', 'destination_balance', 'replication_factor', 'num_edges', 'num_vertices'], ['source_balance', 'replication_factor', 'num_edges', 'num_vertices'], ['vertex_balance', 'source_balance', 'replication_factor', 'num_edges', 'num_vertices'], ['des

# 3.2 Start Training

In [7]:
overall_scores = []
PATH_STORE_MODEL = helper.create_results_folder("/home/ubuntu/cephstorage/graph-processing-run-time-prediction-models")
PATH_STORE_SCORES = helper.create_results_folder("results-graph-processing-run-time-prediction")

for USE_ALL_PARTITIONER in [
    True, 
  ]:
  for algo in [
    "cc", 
    "pr",
    "sssp1", 
    "kcoreavg",
    "synthetic1c0",
    "synthetic10c0"
    ]:
    for model_name in [
      #"KNN",
      #"PolyRegression",
      #"XGB", 
      #"RFR", 
      #"SVR",
      "MLP"
      ]:
      print("We start the training with model ({}) for algorithm ({}). We build ONE model for all partitioners: {}".format(model_name, algo, USE_ALL_PARTITIONER))
      if USE_ALL_PARTITIONER:
        training_approach = "single-model-for-all-partitioners"
        # We use all the partitioners for the paper
        _X_train =  X_train[(X_train.partitioner.isin(paper_partitioners)) &(X_train.algorithm == algo)]
        _y_train =  y_train[(y_train.partitioner.isin(paper_partitioners)) &(y_train.algorithm == algo)]
        _X_test =  X_test[(X_test.partitioner.isin(paper_partitioners)) &(X_test.algorithm == algo)]
        _y_test =  y_test[(y_test.partitioner.isin(paper_partitioners)) &(y_test.algorithm == algo)]
     #   print("_X_train with {} samples. {} are expected".format(len(_X_train), NUM_TRAIN_GRAPHS * NUM_PARTITIONERS))
     #   print("_y_train with {} samples. {} are expected".format(len(_y_train), NUM_TRAIN_GRAPHS * NUM_PARTITIONERS))
     #   print("_X_test with {} samples. {} are expected".format(len(_X_test), NUM_TEST_GRAPHS * NUM_PARTITIONERS))
     #   print("_y_test with {} samples. {} are expected".format(len(_y_test), NUM_TEST_GRAPHS * NUM_PARTITIONERS))
        # We get the model which includes the gridsearch and pipeline
        model = helper.get_model(model_name, featuresets)
        # Get scores: Will include that crossvalidation score for the best hyperparameters and also the testscore for the best hyperparameters. 
        # The testscores will be 1) per partitioner and per graph 2) per graph 3) per partitioner 4) overall score
        scores = train(
          model_name=model_name, 
          algorithm=algo, 
          model=model, 
          X_train=_X_train, 
          y_train=_y_train, 
          X_test=_X_test, 
          y_test=_y_test, 
          feature_combinations=featuresets, 
          target=TARGET, 
          store_model=True,
          result_file_name="{}/{}_{}_{}".format(PATH_STORE_MODEL, training_approach, algo, model_name))


        for score in scores:
          score["algorithm"] = algo
          score["training_approach"] = training_approach
        overall_scores += scores
        pd.DataFrame(scores).to_csv("{}/{}_{}_{}".format(PATH_STORE_SCORES, algo, training_approach, model_name))
      else:
        training_approach = "single-model-per-partitioner"
        for partitioner in paper_partitioners:
            _X_train =  X_train[(X_train.partitioner == partitioner) &(X_train.algorithm == algo)]
            _y_train =  y_train[(y_train.partitioner == partitioner)&(y_train.algorithm == algo)]
            _X_test =  X_test[(X_test.partitioner == partitioner)&(X_test.algorithm == algo)]
            _y_test =  y_test[(y_test.partitioner == partitioner)&(y_test.algorithm == algo)]
        #    print("_X_train with {} samples. {} are expected".format(len(_X_train), NUM_TRAIN_GRAPHS ))
        #    print("_y_train with {} samples. {} are expected".format(len(_y_train), NUM_TRAIN_GRAPHS ))
        #    print("_X_test with {} samples. {} are expected".format(len(_X_test),  NUM_TEST_GRAPHS))
        #    print("_y_test with {} samples. {} are expected".format(len(_y_test), NUM_TEST_GRAPHS))
            # We get the model which includes the gridsearch and pipeline
            model = helper.get_model(model_name, featuresets)
            # Get scores: Will include that crossvalidation score for the best hyperparameters and also the testscore for the best hyperparameters. 
            # The testscores will be 1) per partitioner and per graph 2) per graph 3) per partitioner 4) overall score
            scores = train(
              model_name=model_name, 
              algorithm=algo , 
              model=model, 
              X_train=_X_train, 
              y_train=_y_train, 
              X_test=_X_test,
              y_test=_y_test, 
              feature_combinations=featuresets,
              target=TARGET,
              store_model=True,
              result_file_name="{}/{}_{}_{}_{}".format(PATH_STORE_MODEL, partitioner, training_approach, algo, model_name))
            for score in scores:
              score["algorithm"] = algo
              score["training_approach"] = training_approach
            overall_scores += scores
            pd.DataFrame(scores).to_csv("{}/{}_{}_{}_{}".format(PATH_STORE_SCORES, training_approach, algo, partitioner, model_name))

scores_df = pd.DataFrame(overall_scores)
scores_df.to_csv("{}/scores-{}".format(PATH_STORE_SCORES, TARGET))

We start the training with model (PolyRegression) for algorithm (cc). We build ONE model for all partitioners: True
Trainer will store model in  /home/ubuntu/cephstorage/graph-processing-run-time-prediction-models/2022-12-12-17:41:37/single-model-for-all-partitioners_cc_PolyRegression
Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    2.6s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1960s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    2.8s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1488s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:    3.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1647s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 244 tasks      | elapsed: 

PARAMETERS dict_keys(['cv', 'error_score', 'estimator__check_inverse', 'estimator__func', 'estimator__inverse_func', 'estimator__regressor__memory', 'estimator__regressor__steps', 'estimator__regressor__verbose', 'estimator__regressor__feature_selection', 'estimator__regressor__scaler', 'estimator__regressor__poli', 'estimator__regressor__regressor', 'estimator__regressor__feature_selection__features', 'estimator__regressor__scaler__copy', 'estimator__regressor__scaler__with_mean', 'estimator__regressor__scaler__with_std', 'estimator__regressor__poli__degree', 'estimator__regressor__poli__include_bias', 'estimator__regressor__poli__interaction_only', 'estimator__regressor__poli__order', 'estimator__regressor__regressor__copy_X', 'estimator__regressor__regressor__fit_intercept', 'estimator__regressor__regressor__n_jobs', 'estimator__regressor__regressor__normalize', 'estimator__regressor', 'estimator__transformer', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 're

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    8.1s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   11.4s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   15.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   16.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   19.0s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   21.0s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

PARAMETERS dict_keys(['cv', 'error_score', 'estimator__check_inverse', 'estimator__func', 'estimator__inverse_func', 'estimator__regressor__memory', 'estimator__regressor__steps', 'estimator__regressor__verbose', 'estimator__regressor__feature_selection', 'estimator__regressor__scaler', 'estimator__regressor__regressor', 'estimator__regressor__feature_selection__features', 'estimator__regressor__scaler__copy', 'estimator__regressor__scaler__with_mean', 'estimator__regressor__scaler__with_std', 'estimator__regressor__regressor__activation', 'estimator__regressor__regressor__alpha', 'estimator__regressor__regressor__batch_size', 'estimator__regressor__regressor__beta_1', 'estimator__regressor__regressor__beta_2', 'estimator__regressor__regressor__early_stopping', 'estimator__regressor__regressor__epsilon', 'estimator__regressor__regressor__hidden_layer_sizes', 'estimator__regressor__regressor__learning_rate', 'estimator__regressor__regressor__learning_rate_init', 'estimator__regressor__r

...

PARAMETERS dict_keys(['cv', 'error_score', 'estimator__check_inverse', 'estimator__func', 'estimator__inverse_func', 'estimator__regressor__memory', 'estimator__regressor__steps', 'estimator__regressor__verbose', 'estimator__regressor__feature_selection', 'estimator__regressor__scaler', 'estimator__regressor__regressor', 'estimator__regressor__feature_selection__features', 'estimator__regressor__scaler__copy', 'estimator__regressor__scaler__with_mean', 'estimator__regressor__scaler__with_std', 'estimator__regressor__regressor__activation', 'estimator__regressor__regressor__alpha', 'estimator__regressor__regressor__batch_size', 'estimator__regressor__regressor__beta_1', 'estimator__regressor__regressor__beta_2', 'estimator__regressor__regressor__early_stopping', 'estimator__regressor__regressor__epsilon', 'estimator__regressor__regressor__hidden_layer_sizes', 'estimator__regressor__regressor__learning_rate', 'estimator__regressor__regressor__learning_rate_init', 'estimator__regressor__r

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0317s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0317s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Done  19 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Done  30 tasks      | elapsed:    0.3s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.1988s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Done  68 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 120 tasks      | elapsed:    0.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 255 tasks      | elapsed:    0.9s
[Parallel(n_jobs=-1)]: Done 289 out of 320 | elapsed:    0.9s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done 320 out of 320 | elapsed:    1.0s finished


PARAMETERS dict_keys(['cv', 'error_score', 'estimator__check_inverse', 'estimator__func', 'estimator__inverse_func', 'estimator__regressor__memory', 'estimator__regressor__steps', 'estimator__regressor__verbose', 'estimator__regressor__feature_selection', 'estimator__regressor__scaler', 'estimator__regressor__poli', 'estimator__regressor__regressor', 'estimator__regressor__feature_selection__features', 'estimator__regressor__scaler__copy', 'estimator__regressor__scaler__with_mean', 'estimator__regressor__scaler__with_std', 'estimator__regressor__poli__degree', 'estimator__regressor__poli__include_bias', 'estimator__regressor__poli__interaction_only', 'estimator__regressor__poli__order', 'estimator__regressor__regressor__copy_X', 'estimator__regressor__regressor__fit_intercept', 'estimator__regressor__regressor__n_jobs', 'estimator__regressor__regressor__normalize', 'estimator__regressor', 'estimator__transformer', 'estimator', 'iid', 'n_jobs', 'param_grid', 'pre_dispatch', 'refit', 're

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.5s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    7.3s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    8.8s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   10.0s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   11.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   17.1s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   19.2s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   21.6s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

PARAMETERS dict_keys(['cv', 'error_score', 'estimator__check_inverse', 'estimator__func', 'estimator__inverse_func', 'estimator__regressor__memory', 'estimator__regressor__steps', 'estimator__regressor__verbose', 'estimator__regressor__feature_selection', 'estimator__regressor__scaler', 'estimator__regressor__regressor', 'estimator__regressor__feature_selection__features', 'estimator__regressor__scaler__copy', 'estimator__regressor__scaler__with_mean', 'estimator__regressor__scaler__with_std', 'estimator__regressor__regressor__activation', 'estimator__regressor__regressor__alpha', 'estimator__regressor__regressor__batch_size', 'estimator__regressor__regressor__beta_1', 'estimator__regressor__regressor__beta_2', 'estimator__regressor__regressor__early_stopping', 'estimator__regressor__regressor__epsilon', 'estimator__regressor__regressor__hidden_layer_sizes', 'estimator__regressor__regressor__learning_rate', 'estimator__regressor__regressor__learning_rate_init', 'estimator__regressor__r

# 4. Evaluations
Have a look at the run-time-prediction-scores.ipynb with the scores reported in the paper 

set the right timestamp!

In [8]:
scores_df = pd.read_csv("{}/scores-processing_time".format(PATH_STORE_SCORES))

In [9]:
scores_overall_df = scores_df[(scores_df.graph == "all") & (scores_df.partitioner == "all") & (scores_df.training_approach == "single-model-for-all-partitioners")  ]
print("We trained {} models but select the best one based on crossvalidation on the rmat graphs".format(len(scores_overall_df)))

idx = scores_overall_df.groupby(["algorithm", "partitioner"])['rmat_cv_score'].transform(max) == scores_overall_df['rmat_cv_score']
best_scores_overall_df = scores_overall_df[idx].sort_values(by=["algorithm", "mape"])
print("One average we can predict the grap processing run-time with a mape of {}".format(best_scores_overall_df["mape"].mean()))
best_scores_overall_df[["r2", "rmse", "mape", "partitioner", "graph", "model", "algorithm", "rmat_cv_score", "training_approach", "rmat_best_params"]]

We trained 12 models but select the best one based on crossvalidation on the rmat graphs
One average we can predict the grap processing run-time with a mape of 0.315802392250061


,r2,rmse,mape,partitioner,graph,model,algorithm,rmat_cv_score,training_approach,rmat_best_params
88,0.828745,35304.401606,0.272462,all,all,PolyRegression,cc,-0.199110,single-model-for-all-partitioners,"{'regressor__feature_selection__features': ['vertex_balance', 'destination_balance', 'source_balance', 'replication_factor', 'edge_balance', 'num_edges', 'num_vertices'], 'regressor__poli__degree': 2}"
622,0.567524,167424.679961,0.370609,all,all,PolyRegression,kcoreavg,-0.368741,single-model-for-all-partitioners,"{'regressor__feature_selection__features': ['vertex_balance', 'destination_balance', 'source_balance', 'replication_factor', 'edge_balance', 'num_edges', 'num_vertices'], 'regressor__poli__degree': 2}"
266,0.680508,9054.331592,0.313933,all,all,PolyRegression,pr,-0.162552,single-model-for-all-partitioners,"{'regressor__feature_selection__features': ['destination_balance', 'source_balance', 'replication_factor', 'edge_balance', 'num_edges', 'num_vertices'], 'regressor__poli__degree': 2}"
444,0.916665,59542.297729,0.407423,all,all,PolyRegression,sssp1,-0.141117,single-model-for-all-partitioners,"{'regressor__feature_selection__features': ['vertex_balance', 'source_balance', 'replication_factor', 'edge_balance', 'num_edges', 'num_vertices'], 'regressor__poli__degree': 2}"
978,0.785240,14193.292422,0.259128,all,all,PolyRegression,synthetic10c0,-0.181267,single-model-for-all-partitioners,"{'regressor__feature_selection__features': ['destination_balance', 'source_balance', 'replication_factor', 'edge_balance', 'num_edges', 'num_vertices'], 'regressor__poli__degree': 2}"
800,0.742168,13227.444964,0.271258,all,all,PolyRegression,synthetic1c0,-0.175891,single-model-for-all-partitioners,"{'regressor__feature_selection__features': ['destination_balance', 'source_balance', 'replication_factor', 'edge_balance', 'num_edges', 'num_vertices'], 'regressor__poli__degree': 2}"
